In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
from pandarallel import pandarallel

In [2]:
# turn on dataframe parallel feature.
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
train = pd.read_csv(r'./Datasets/train.csv', 
                 dtype={'TripType': object, 
                        'VisitNumber': object,
                        'Weekday': object,
                        'Upc': object,
                        'ScanCount': int,
                        'DepartmentDescription': object,
                        'FinelineNumber': object}
                )
test = pd.read_csv(r'./Datasets/test.csv', 
                 dtype={'TripType': object, 
                        'VisitNumber': object,
                        'Weekday': object,
                        'Upc': object,
                        'ScanCount': int,
                        'DepartmentDescription': object,
                        'FinelineNumber': object}
                )
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647054 entries, 0 to 647053
Data columns (total 7 columns):
TripType                 647054 non-null object
VisitNumber              647054 non-null object
Weekday                  647054 non-null object
Upc                      642925 non-null object
ScanCount                647054 non-null int64
DepartmentDescription    645693 non-null object
FinelineNumber           642925 non-null object
dtypes: int64(1), object(6)
memory usage: 34.6+ MB


# EDA

After loading the training dataset, there are only 5 features for us to construct a prediction model. In order to distill more information to increase accuracy, we should get more insight from the original dataset.

## UPC

UPC(Universal Product Code) is a barcode symbology that can identify the products. According to this [link](https://www.gs1-us.info/upc-codes/), we know UPC consists of 12 numeric digits that are composed by company prefix and item reference. My purpose is to correct the UPC of the dataset and extract information(such as price, product...etc.) to get more features for predicting.

### Adjust UPC

This [webpage](https://support.suredone.com/support/solutions/articles/1000254202-walmart-missing-required-field-upc) interprets accepted identifiers that Walmart is using (UPC 12 digits). We can calculate the check digit followed by [this rule](http://www.azaleabarcodes.com/white-papers/upc-barcode-check-digit/).

We notice there are some 12-digit values that do not follow check digit rule. ([Selling page](https://www.walmart.ca/en/ip/simply-canvas-panel-18-x-24-3p-8-x-10/6000111967712) on Walmart for UPC 501138607481). Since it is hard to find a easy way correcting them. I'll fill them by the average value of VisitNumber.

In [5]:
train['Upc'].str.len().value_counts().sort_index()

3.0          7
4.0      29745
5.0        372
7.0          1
8.0        412
9.0       2166
10.0    433341
11.0    168418
12.0      8463
Name: Upc, dtype: int64

In [6]:
train.loc[train['Upc'].str.len()==12, 'Upc']

48        501138607481
50        501138610338
58        692656840005
60        695334428094
67        692249579995
              ...     
646730    896110116858
646771    489704605078
646788    695334428112
646893    471286236459
646921    361422003835
Name: Upc, Length: 8463, dtype: object

In [7]:
## Calculate check digit number
def fillCheckNumber(upc):
    if len(upc) <= 11:
        upc = upc.zfill(11)
        oddNum = sum([int(n) for n in upc[0::2]])*3
        evenNum = sum([int(n) for n in upc[1::2]])
        checkNum = 10 - (oddNum + evenNum) % 10
        return upc + str(checkNum) if checkNum != 10 else upc +'0'
    else:
        return 'Unidentified'

In [8]:
train['Upc_check_number'] = train['Upc'].fillna('Unidentified').parallel_apply(fillCheckNumber)
test['Upc_check_number'] = test['Upc'].fillna('Unidentified').parallel_apply(fillCheckNumber)

In [9]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Upc_check_number
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000,681131529297
1,30,7,Friday,60538815980,1,SHOES,8931,605388159809
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504,074108110992
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565,022384035102
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017,020066137441


### Crawl data from BrickSeek

There is a [Product Lookup API](https://developer.walmartlabs.com/docs) from Walmart which can get an information by UPC. But I do not have a developer account and the registration is currently disabled. So I will crawl manufacturer's suggested retail price from [BrickSeek](https://brickseek.com).

In [10]:
import requests
import random
import re
import asyncio
import aiohttp
import nest_asyncio
import multiprocessing as mp
from bs4 import BeautifulSoup

In [11]:
## Get proxies from free proxy website
def catchProxyList():
    proxy_ips = []
    res = requests.get(r'https://free-proxy-list.net/', verify = False)
    soup = BeautifulSoup(res.text)

    for tr_item in soup.find('tbody').find_all('tr'):
        ip = ':'.join([tr_item.find_all('td')[0].text, tr_item.find_all('td')[1].text])
        proxy_ips.append(ip)
    return proxy_ips
proxy_ips = catchProxyList()
proxy_ips[:5]

/Users/jude/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'free-proxy-list.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


['200.73.128.5:8080',
 '41.217.217.60:47981',
 '94.177.245.244:8080',
 '200.89.178.177:80',
 '170.254.176.113:28980']

In [12]:
## There are many dead proxies in the proxy list. We need to check validation on it.
def checkProxyValid(proxy):
    try:
        resp = requests.get('http://icanhazip.com',
                            proxies={'http': 'http://jude:2xuidlji@{}'.format(proxy),
                                  'https': 'http://jude:2xuidlji@{}'.format(proxy)}, 
                            timeout=3)
        if re.findall('\d*.\d*.\d*.\d*', resp.text)[0] == re.findall('\d*.\d*.\d*.\d*', proxy)[0]:
            return True
        else:
            return False
    except:
        return False
    
def getValidIP():
    global proxy_ips
    
    ip = random.choice(proxy_ips)
    
    while not checkProxyValid(ip):
        proxy_ips.remove(ip)
        if not proxy_ips:
            proxy_ips = catchProxyList()
            
        ip = random.choice(proxy_ips)
    return ip

In [13]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

def crawlPrice(upc, headers = headers):
    global ip
    
    fromData = {
        'method': 'upc',
        'sku': '',
        'upc': upc,
        'zip': '',
        'sort': 'recommended'
    }
    
    try:
        resp = requests.post('https://brickseek.com/walmart-inventory-checker/', 
                         data = fromData,
                         headers = headers,
                         proxies={'http': 'http://jude:2xuidlji@{}'.format(ip),
                                  'https': 'http://jude:2xuidlji@{}'.format(ip)},
                         timeout= 5 
                            )
    except:
        ip = getValidIP()
        print('change ip')
        return crawlPrice(upc)
    
    soup = BeautifulSoup(resp.text)
    try:
        result = soup.find('strong', string = 'MSRP:').find_next_sibling().get_text()
        print('success')
    except:
        result = 'N/A'
        print('error item')
    return {upc: result}

In [14]:
if __name__ == '__main__':
    upcList = set(train['Upc_check_number']).union(set(test['Upc_check_number']))
    upcList.remove('Unidentified')
    ip = getValidIP()
    
    pool = mp.Pool(4)                       # number strongly affected
    crawl_jobs = [pool.apply_async(crawlPrice, args=(upc,)) for upc in upcList]
    upc_result = [j.get() for j in crawl_jobs]

change ip
error item
change ip
success
success
error item
success
success
change ip
success
change ip


KeyboardInterrupt: 

change ip
success
success


In [56]:
import pickle

In [ ]:
pickle.dump(crawl_jobs, 'crawl_jobs.pkl')
pickle.dump(upc_result, 'crawl_jobs.pkl')

In [18]:
pool.terminate()

***

In [ ]:
nest_asyncio.apply()

upcList = set(train['Upc_check_number']).union(set(test['Upc_check_number']))

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [loop.create_task(crawlPrice(upc, session)) for upc in upcList]
        finished, unfinished = await asyncio.wait(tasks)
        upc_result = [f.result() for f in finished]
            

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

In [ ]:
async with aiohttp.ClientSession() as session:
    ip = random.choice(proxy_ips)
    await checkProxyValid(ip, session)

***

In [ ]:
df.loc[df.duplicated(keep=False)].groupby('VisitNumber').size().argmax()

In [ ]:
df['ScanCount'].max()

In [ ]:
df.loc[df['VisitNumber'] == '44764']

In [ ]:
(df['ScanCount']<0)

In [ ]:
df.loc[df['VisitNumber'].isin(df.loc[df['ScanCount']<0]['VisitNumber'])].sort_values('VisitNumber')

In [ ]:
pandas_profiling.ProfileReport(train)